# Ejercicio OpenStreetMap
Para este ejercicio vas a trabar con Overpass, que es la API que utiliza los datos de OpenStreetMap. Tras obtener datos geográficos mediante la API, tendrás que representarlos con Folium

Hay que obtener los siguientes datos de cada ubicación conseguida en OpenStreetMap
* tipo de amenity
* latitud
* longitud
* nombre
* direccion

**NOTA**: En este Notebook utilizarás gráficas de Folium, y JupyterLab de momento no lo soporta, a no ser que instales una extensión. Para solucionar esto, abre el Notebook desde Jupyter normal:

![imagen](./img/jupyter.png)

Tienes en [este enlace](https://wiki.openstreetmap.org/wiki/Key:amenity#Healthcare) la lista de *amenities* que vas a usar en el notebook.

## Librerías
Importa las librerias que vamos a usar en el Notebook

In [2]:
import requests
import json
import pandas as pd

Ataca a la API de OpenStreetmap con el siguiente código de ejemplo

In [3]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="ice_cream"]
  (40.420801, -3.694702,40.423754, -3.688167); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()

<Response [200]>


Con el siguiente código podrás adaptar el JSON que devuelve la API, a un formato DataFrame más amigable

In [4]:
def json_to_df(data):
    elements = data['elements']
    places = {'tipo': [], 'lat': [], 'lon': [], 'name': [], 'address': []}
    
    for i in elements:
        
        tipo = i.get('tags', None).get('amenity', None)
        latitude = i.get('lat', None)
        longitude = i.get('lon', None)
        name = i.get('tags', {}).get('name', "NO NAME")
        street = i.get('tags', {}).get('addr:street', "NO STREET")
        number = i.get('tags', {}).get('addr:housenumber', 9999)

        places['tipo'].append(tipo)
        places['lat'].append(latitude)
        places['lon'].append(longitude)
        places['name'].append(name)
        places['address'].append(street + ' ' + str(number))

            
    return pd.DataFrame(places)

Como ejemplo, para obtener todos los restaurantes cercanos a The Bridge

In [5]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="restaurant"]
  (40.420801, -3.694702,40.423754, -3.688167); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df.head()

<Response [200]>


,tipo,lat,lon,name,address
0,restaurant,40.423086,-3.693547,La Castafiore,NO STREET 9999
1,restaurant,40.422607,-3.693256,Sake Dining Himawari,NO STREET 9999
2,restaurant,40.423315,-3.692506,Ainhoa,Calle de Doña Bárbara de Braganza 12
3,restaurant,40.421960,-3.694270,La Kitchen,NO STREET 9999
4,restaurant,40.421956,-3.693905,En La Gloria,Calle de Prim 9


In [6]:
import folium


def pinta_mapa(df, center_lat, center_lon, zoom):
    # map
    map_restaurants = folium.Map(location=[center_lat, center_lon], zoom_start=zoom)

    df.apply(lambda row: folium.Marker(location=[row['lat'], row['lon']],
                                             popup=row['name']).add_to(map_restaurants), axis=1)

    # display map
    return map_restaurants

pinta_mapa(df, 40.422081, -3.690748, 17)

## Ejercicio 1
### Búsqueda de aparcamiento
Para este caso de uso imagina que vas a Madrid de viaje y estás buscando posibles parkings donde dejar el coche, crea un DataFrame con todos los parkings (de OpenStreetMap) de la ciudad de Madrid. Para ello define una cuadrícula con dos coordenadas sobre Madrid.

Representa en un mapa los resultados

In [7]:
overpass_query = """
[out:json];
node["amenity"="parking"]
  (40.396013, -3.746999, 40.480011, -3.625910); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [200]>


,tipo,lat,lon,name,address
0,parking,40.426712,-3.714370,Princesa 25,NO STREET 9999
1,parking,40.414469,-3.703549,Parking Benavente,NO STREET 9999
2,parking,40.406444,-3.708171,NO NAME,NO STREET 9999
3,parking,40.416691,-3.709364,NO NAME,NO STREET 9999
4,parking,40.447447,-3.627856,NO NAME,NO STREET 9999
...,...,...,...,...,...
247,parking,40.408562,-3.669664,NO NAME,NO STREET 9999
248,parking,40.407339,-3.669269,NO NAME,NO STREET 9999
249,parking,40.405968,-3.666643,NO NAME,NO STREET 9999
250,parking,40.455416,-3.678707,NO NAME,Calle de Oruro 12


In [9]:
pinta_mapa(df, 40.407682, -3.668754, 12)

## Mudanza
Imagina que te quieres mudar al Berrueco, pero antes te gustaría obtener algún indicador de si hay civilización por esa zona. Mira a ver si existe gasolineras y/o colegios por la zona. Define, al igual que en el caso anterior, un rectángulo mediante dos coordenadas. Busca la manera de realizar una única petición a la API.

Representa después en un mapa, por color los colegios y gasolineras

In [8]:
overpass_query = """
[out:json];
node["amenity"="fuel|school"]
  (40.851838, -3.627774, 40.941457, -3.481755);
out;
node["amenity"="school"]
  (40.851838, -3.627774, 40.941457, -3.481755);
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [200]>


,tipo,lat,lon,name,address
0,school,40.867720,-3.614696,Escuela infantil de La Cabrera,NO STREET 9999
1,school,40.855890,-3.483306,Patones,NO STREET 9999
2,school,40.872876,-3.606421,La Cabrera,NO STREET 9999
3,school,40.864566,-3.614058,Eq. Aten.Temprana La Cabrera-Lozoyuela,NO STREET 9999
4,school,40.872395,-3.608802,C.A.P. De La Cabrera,NO STREET 9999
5,school,40.863864,-3.615523,Equipo General La Cabrera,NO STREET 9999
6,school,40.872395,-3.608802,C.A.P. De La Cabrera,NO STREET 9999


In [21]:
pinta_mapa(df, 40.863864, -3.615523, 12)

## Zonas COVID
Imagina que trabajas en el ayuntamiento de tu barrio/pueblo/ciudad, y por motivos del COVID tienes que identificar:
1. Todas las zonas abiertas de uso común (zonas de barbacoas, fuentes de agua y parques infantiles (kindergarten)), para cerrarlas.
2. Todos los hospitales y clínicas de la zona.

Como no sabes muy bien qué puntos serán, utilizas la API de OpenStreetMap. Representa todo en un único mapa

In [22]:
overpass_query = """
[out:json];
node["amenity"="bbq"]
  (40.430292, -3.829166, 40.465285, -3.764212);
out;
node["amenity"="drinking_water"]
  (40.430292, -3.829166, 40.465285, -3.764212);
out;
node["amenity"="hospital"]
  (40.430292, -3.829166, 40.465285, -3.764212);
out;
node["amenity"="clinic"]
  (40.430292, -3.829166, 40.465285, -3.764212);
out;
node["amenity"="kindergarten"]
  (40.430292, -3.829166, 40.465285, -3.764212);
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [200]>


,tipo,lat,lon,name,address
0,drinking_water,40.436855,-3.819278,Fuente de la Poza,NO STREET 9999
1,drinking_water,40.440148,-3.765316,NO NAME,NO STREET 9999
2,drinking_water,40.454084,-3.808987,NO NAME,NO STREET 9999
3,drinking_water,40.447745,-3.789207,NO NAME,NO STREET 9999
4,drinking_water,40.449560,-3.787557,NO NAME,NO STREET 9999
...,...,...,...,...,...
112,kindergarten,40.454846,-3.795735,Mi Pequeña Granja,NO STREET 9999
113,kindergarten,40.461235,-3.787613,Pentapolín,NO STREET 9999
114,kindergarten,40.460623,-3.783490,Alaria,NO STREET 9999
115,kindergarten,40.460337,-3.783349,El Osito de Papel,NO STREET 9999


In [24]:
pinta_mapa(df, 40.447745, -3.789207, 14)

## Supermercados
Para este ejemplo supondrás que trabajas en una cadena de supermercados en la que se está planteando construir un nuevo local en la provincia de Sevilla. Busca mediante la API de Overpass todos los supermercados de la provincia para poder comprobar en qué nucleos urbanos habría poca competencia.

En este caso hay que buscar por área, no por cuadrícula! Investiga en la documentación/Stackoverflow cómo se haría eso.

In [10]:
overpass_url

'http://overpass-api.de/api/interpreter'

In [25]:
overpass_query = """
[out:json];
area[name="Sevilla"];
nwr[shop=supermarket](area);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [200]>


,tipo,lat,lon,name,address
0,None,37.373002,-6.047204,MAS,NO STREET 9999
1,None,37.390016,-5.973769,Maxi Dia,NO STREET 9999
2,None,36.994532,-5.572320,Dia,NO STREET 9999
3,None,37.406364,-5.976342,El Jamón,NO STREET 9999
4,None,37.406508,-5.973342,Valdajimena,Avenida de San Juan de la Salle 9999
...,...,...,...,...,...
466,None,NaN,NaN,Super Carmela,NO STREET 9999
467,None,NaN,NaN,Aldi,Avenida Doctor Villa 1
468,None,NaN,NaN,Mercadona,NO STREET 9999
469,None,NaN,NaN,Spar,Calle Pío XII 1


In [26]:
df['tipo'] = df['tipo'].fillna("supermarket")
df = df.dropna()

In [30]:
pinta_mapa(df, 36.994532, -5.572320, 8)